# Developing a model architecture for segmentation

## Packages to use

In [ ]:
## Imports go here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import nibabel as nib
from sklearn.model_selection import GridSearchCV
import scikeras